# Polygenic Risk Scores

Before we can jump into polygenic risk scores (PRS) we must first cover a few prerequisite topics:

## What is a genome wide association study

New sequencing technologies like SNP array sequencing have made DNA sequencing cost less than $100/person. This has led to companies like 23andMe performing personal sequencing tests and governmental agencies developing biobanks with hundreds of thousands of genomic samples which we can use to discover associations between certain genomic variants and traits. A Genome-wide association study (GWAS) then is just a determining if there is a relationship between every variant and a trait of interest (assuming that each variant has only additive interactions which is true enough for now). 

We can visualize this as a regression on each trait as shown below. Then we perform a statistical test to determine whether or not the slope of the line is non-zero; this would indicate that there is a relationship between the variant and trait of interest. Concretely, given the regression line for a single trait $Y=\beta_{1}*X+\beta_{2}$ where $\beta_{1} and \beta_{2}$ correspond to the slope and constant in our regression and $X$ is the number of copies of that variant are in an individuals genome. Under typical circumstances $X$ should always be 0, 1, or 2. Once we repeat this regression for each variant in a genome we've performed a GWAS. Collectively these $\beta$ values are called the *summary statistics* of a GWAS.

![linear regression on a given trait](imgs/regression.jpg)

The most commonly visualized output of a GWAS are the p-values from these association tests. If we plot the $-log_{10}(p-value)$ against the location of every variant on the genome we get a *Manhattan Plot*, named for the visible clusters of significant p-values that might emulate a sky-scraper (or, for especially polygenic traits, the entire plot might resemble a city skyline). 

![example manhattan plot](imgs/Manhattan_Plot.png)

## Why do the significant p-values cluster in manhattan plots?

A crossing over event occurs when a portion of one chromatid exchanges with its equivalent portion of a sister chromatid. Portions of the genome that are closer to each other are more likely to be involved in the same crossing over event, causing them to be inherited together more often. This effect compounded over many, many generations causes variants in close genomic proximity to almost always be inherited together, this effect is called *linkage disequilbrium* or LD. We can compute a numerical quantity for LD within a ancestral population group as the proportion within a large sample that two variants are inherited together. 

This effect also explains the towers we see in manhattan plots. If there a single variant that actually causes some trait it will be highly correlated in our GWAS study, but due to LD every other variant in close proximity will also be highly correlated. Finding the causal variant (or variants) within a tower is more complex than choosing the smallest p-value and is an area of active research called *finemapping* with the standard tool for doing so being called `susie`. 

## What are polygenic risk scores?

We previously thought about GWAS as many linear regressions, a natural next step might be predicting some value given an input set of variants. Say those variants correspond to the actual genome of a person. What might this value mean? For a binary trait, a higher value could indicate that this person is more likely to possess the trait, similarly a lower score could indicate that this person is less likely to possess the trait. If that trait is a disease, then a higher score could indicate that this person is at higher risk for the disease, therefore we call these predicted values *polygenic risk scores* though they can be calculated for theoretically any trait which has a corresponding GWAS and $\beta$ values available. 

Some readers might have already realized that PRS are not between 0 and 1 as is requred of all probabilities. It's important to recognize that PRS are relative values of risk and only make sense in the context of a distribution of both healthy and diseased individuals and their corresponding PRS. This is just one of several important caveats from any PRS.

In a linear regression we predict our output $Y$ by plugging our input $X$ into our calculated formula:
$$ Y = \beta_{1} * X + \beta_{2} $$

In GWAS we have one input $X \in \{0, 1, 2\}$ for each variant whose value depends on the number of copies of that variant in the sample. For $n$ possible variants, our simplified regression formula becomes:
$$ PRS = \sum_{i=1}^{n} \beta_{i,1} * X_{i} + \beta_{i,2} $$ 

Note that we assume genes are additively related to our trait which we know is incorrect but is still a useful model. We are also ignoring covariates but actual tools will use a Generalized Mixed Linear Models to capture their effect.

## Why can't we trust polygenic risk scores?

The main difference between a polygenic risk score and a regular linear regression is that instead of using one variable X to predict an outcome Y. We are using hundreds of thousands of variables, or genomic variants, to predict one outcome phenotype.

But just like there are many caveats to simply interpreting a straight line where it may not exist, we can't just take a PRS at face value without checking several assumptions:

1. We cannot predict outside the sample data. If we performed GWAS in a population of all European ancestries and then use the $\beta$ values for a PRS of an individual of African descent, our results wouldn't make any sense. If we only trained on a single population we can only predict PRS inside that same population. This is because some variants may have been extremely rare or even non-existant inside our GWAS population and predicting values using those variants could badly skew our results.
2. Because we are performing **linear** regressions at each variant, we are assuming that having two copies of a variant will have twice the effect on our trait as having a single copy. While this is sometimes true, it certainly not a rule and there are many known traits where a second copy will have diminishing or increased returns.
3. We also assume homoscedasticity, or that even if a variant does explain a trait, having 0, 1, or 2 copies of a variant explains that trait equally. It could be entirely possible that 0 copies of a variant is a sufficient condition for a trait, but not mandatory. Then we can conclude information from 0 copies of that trait but we conclude less information from other values.
4. Each variant may not be independent. In fact we're sure that some variants are more likely to be inherited together because of LD. We'll try to correct this through a process called clumping when calculating PRS. Independence can also be violated by non-linear relationships between variants.
5. If our input sample does not have the exact same variants and covariates as our GWAS any PRS we generate will not include all the information and cannot be relied upon.
6. Finally, our trait of interest may not even be fully or even majority explained by genetics. Many traits are environmentally dependant and although we can perform GWAS and generate PRS for these traits. Relying on these outcomes is dubious at best.

We must always make sure to account for these errors where possible and acknowledge that our results contain biases and caveats when we cannot. Often, especially when predicting outside of our training population, the caveats are so great we cannot rely on a PRS score at all and it could be inherently harmful to do so. For example incorrectly telling someone they are at great risk for a disease could cost them decades of expensive yet unneccessary preventative treatments. Similarly, informing someone they are not at risk for a certain disease might discourage them from seeking treatment despite seeing warning signs. Although these scores are desceptively simple to calculate, the risks are real and have the potential to be devistating.

## So then are polygenic risk scores useful?

Despite all their caveats and risks, PRS can be an incredibly useful tool for a trained clinitian. If a PRS score is calculated for keeping in mind the aforementioned caveats, it could be inform a patient that they are at a higher risk for a disease which may they may not have tested for beforehand, potentially saving lives. Although this practice is not yet common, it is the future envisioned by *precision medicine* and PRS may play a role in its eventual reality.

## How do we actually calculate a polygenic risk score?

### Data

We first need an individual's set of variants. Researchers typically grab these from large biobanks like All of Us or UK Biobank but usually personal genomic tests are sufficient like Ancestry.com or 23andMe. The theoretical best use case though is from clinitians who can order genomic tests.

We also need the summary statistics from a previously done GWAS. Fortunately, for every published GWAS paper these are almost always publically available at an online database called the [GWAS catalog](https://www.ebi.ac.uk/gwas/downloads/summary-statistics).

Finally we need LD values for your population of interest. It's best to use the identical LD values to the GWAS which you took the summary statistics from. But valid LD scores can also be downloaded for many populations online. The 1000 Genomes dataset for example provides LD values that work well for a population of European descent.

### Tool

Although the calculation is simple enough that we could theoretically write code to parse the neccessary files and calculate a PRS ourselves, we prefer standardized, efficient, and reliable tools.

The current state of the art is called plink (though Hail is a newer tool that may soon replace it) and its binaries for all common devices can be downloaded on [their website](https://www.cog-genomics.org/plink/2.0/)

Make sure to add plink to your path or call the binary directly. To verify that it's installed properly execute `plink --version` and check for a message similar to the one below:
```
PLINK v1.90b7.2 64-bit (11 Dec 2023)
```

Version 1.9 or 2.0 will not make a substantial difference in calcualting PRS.

### Commands

First we clump our variants to help account for LD. This chooses the most representative variant in a cluster of representative variants if they are in close enough LD. Here it's not incredibly important that we find the actual causal variant because their correlations with the trait of interest will be very similar. We only want to make sure that we don't over represent the contribution from one significant variant by including all of its highly correlated variants as well.

The input variant tag will depend on the file type. Common versions are VCF or `.ped .bim .fam` collectively called plink binary formats.
```
plink --clump <input variant tag and file name> --clump-p2 <SP2 column p-value threshold> --clump-r2 <r^2 threshold in the linear regression> --clump-kb <clump kilobase radius>
```

Finally, assuming you've performed all the neccessary legwork, actually generating a PRS is simple:
```
plink <input data tag and file name> --score <summary statistics file>
```